# Get BBC Dataset

In [0]:
from google.colab import drive
drive.mount("./gdrive")

Drive already mounted at ./gdrive; to attempt to forcibly remount, call drive.mount("./gdrive", force_remount=True).


In [0]:
cd gdrive/My\ Drive/Colab\ Notebooks/Datasets/Text\ Summarization

/content/gdrive/My Drive/Colab Notebooks/Datasets/Text Summarization


In [0]:
import pickle 
with open('myfile.pkl','rb') as fp:
  heading, article = pickle.load(fp)

**Data from BBC News Dataset Loaded. The variables are *heading* and *article***

# Preprocess the Data , remove the \n\n and \xa0 , lower case everything  (stemming/lemmatization advantageous? Try it out later)

## I think I should keep the comma's in for now , maybe the Attention mechanism later would find words near a comma a good place to give attention to

In [0]:
heading = [h.lower() for h in heading]
article = [a.lower() for a in article]

In [0]:
article = [a.replace('\n','').replace('\xa0',' ') for a in article]

# Toeknize the data , Make Word Embeddings from GloVe

## Mainly what I have to do is make a matrix of size (vocab_size, 100) which I can initialize as my weight matrix. Be careful, for now I am assuming no new words can appear in a text other than the ones I have seen before , so later if this works improve it to take in other words too

In [0]:
word_dimensions = 100
import pandas as pd
df = pd.read_csv('glove.6B.100d.txt', sep=" ", quoting=3, header=None, index_col=0)

In [0]:
glove = {key: val.values for key, val in df.T.items()}

In [0]:
import nltk
from collections import Counter
import numpy as np

In [0]:
words = []
tokenizer = nltk.tokenize.WordPunctTokenizer()
for a in article:
  tokenized_text = tokenizer.tokenize(a)
  for tokens in tokenized_text:
    words.append(tokens)
    
for h in heading:
  tokenized_text = tokenizer.tokenize(h)
  for tokens in tokenized_text:
    words.append(tokens)

In [0]:
wordcounts = Counter(words).most_common(170000)
vocab_list = np.array([word for word,_ in wordcounts])
dictionary_word_code = {word:code for code,word in enumerate(vocab_list)}

In [0]:
embedding_list = []
i = 0
while i != vocab_list.shape[0] :
  try:
    embedding_list.append(glove[vocab_list[i]])
    i = i+1
  except KeyError as e:
    embedding_list.append(np.random.normal(0,0.1,[word_dimensions]))
    i = i+1
    
  if i%1000 == 0:
    print(i)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [0]:
embedding_matrix = np.vstack(embedding_list)

**Embedding Matrix is created and in the variable *embedding_matrix***

# Making a Bi-Directional RNN with Attention , no dropout layer

## For now making a 3 Layer LSTM deep network (could cause vanishing gradient problem) and not using the EOS thing, will make the summary a fixed number of words , mostly wont be coherent but lets see if any output comes at all first.

In [0]:
import tensorflow as tf

#### 'The' also has a code of 0 and you are gonna zero pad things so could lead to problems

#### Also you are using too much RAM afterwards change variable names so you overwriting them and not keep making new ones
Ok so for saving RAM I am thinking I will make a pickle file for all the important stuff and load it in so all the useless variables dont take up space

In [0]:
tokenized_articles = []
tokenized_headings = []
for a in article:
  tokens = tokenizer.tokenize(a)
  tokenized_articles.append(tokens)
for h in heading:
  tokens = tokenizer.tokenize(h)
  tokenized_headings.append(tokens)
  
max_length_article = max([len(a) for a in tokenized_articles])
max_length_heading = max([len(h) for h in tokenized_headings])

numerical_articles = []
numerical_headings = []
for a in tokenized_articles:
    list1 = [dictionary_word_code[b] for b in a]
    numerical_articles.append(list1)
for h in tokenized_headings:
  list1 = [dictionary_word_code[b] for b in h]
  numerical_headings.append(list1)
  
padded_numerical_articles = []
padded_numerical_headings = []
for a in numerical_articles:
  zeros = [0]*(max_length_article - len(a))
  list1 = a + zeros
  padded_numerical_articles.append(list1)
for h in numerical_headings:
  zeros = [0]*(max_length_heading - len(h))
  list1 = h + zeros
  padded_numerical_headings.append(list1)
  
with open("input_data.pickle", "wb") as f:
    pickle.dump((embedding_matrix,padded_numerical_articles,padded_numerical_headings,vocab_list), f)

# Too much RAM was used so start from here

In [1]:
import pickle 
with open("input_data.pickle","rb") as f:
  _ , articles , headings, vocab_list = pickle.load(f)

In [2]:
import numpy as np
def next_batch(articles,headings,batch_size,num_time_steps = 850,num_output_steps = 84):
  
  snipped_articles = []
  snipped_headings = []
  
  for a in articles:
    part_article = a[0:num_time_steps]
    snipped_articles.append(part_article)
    
  random = np.random.randint(0,len(articles) - batch_size -1)
  batch_x = np.array(snipped_articles[random:random+batch_size]).reshape([batch_size,num_time_steps,1])
  onehotlistarticles = []
  for article in batch_x:
    for num in article:
      zeros = np.array([0]*len(vocab_list))
      zeros[num] = 1
      onehotlistarticles.append(zeros)
  onehotlistarticles = np.array(onehotlistarticles).reshape([batch_size,num_time_steps,-1])
  
  for h in headings:
    part_heading = h[0:num_output_steps]
    snipped_headings.append(part_heading)
  
  batch_y = np.array(snipped_headings[random:random+batch_size]).reshape([batch_size,num_output_steps,1])
  onehotlistheadings = []
  for heading in batch_y:
    for num in heading:
      zeros = np.array([0]*len(vocab_list))
      zeros[num] = 1
      onehotlistheadings.append(zeros)
  onehotlistheadings = np.array(onehotlistheadings).reshape([batch_size,num_output_steps,-1])
  
  return onehotlistarticles , onehotlistheadings

# TensorFlow Graph 

## Encoder 

In [3]:
def length(sequence):
  used = tf.reduce_max(sequence, 2)
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  return length

**See Imext Notebook for explaination of tf.reduce_max and how axis collapses and stuff**

In [4]:
import tensorflow as tf
import numpy as np

/home/administrator/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
num_time_steps = 850
num_output_steps = 84
dimensions_rnn_cell = 100
batch_size = 3

In [6]:
tf.reset_default_graph()

In [7]:
X = tf.placeholder(tf.float32,shape = [None,num_time_steps,len(vocab_list)])

** Right now too tired, not initializing with Embedding Matrix as its a pain , have no clue what the fuck the function wants from me, come back later here if the performance is not so good , see the documentation for initializer in LSTMCell**

In [8]:
lstm_fw_cell = [tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell),tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell),tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell)]
lstm_bw_cell = [tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell),tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell),tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell)]

outputs, output_state_fw, output_state_bw = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, X, dtype='float32', sequence_length = length(X))

** Cool so session crashes after using all the RAM**

** For testing if this is working or not **

In [10]:
with tf.Session() as sess:
  sess.run(init)
  batch_x, v = next_batch(articles,headings,batch_size)
  a1 = sess.run(outputs,feed_dict = {X:batch_x})

In [17]:
a1.shape

(3, 850, 200)

## Attention Mechanism : For now, wont be taking input from the final output of the network and lets see how it performs, afterwards take as input from the output of the final stage too 

## I think thats possible because the final outputs are also placeholders right? So that can be input to your 'neural_network' function

### Define a Neural Network basically 100 : 50 : 1

In [9]:
combined_BiRNN_outputs = tf.placeholder(tf.float32,shape = [None,num_time_steps,2*dimensions_rnn_cell])

In [10]:
def neural_network(combined_BiRnn_output):
    
    Wx1 = tf.Variable(tf.random_normal([batch_size,200,50],0.3,1/200))
    Wx2 = tf.Variable(tf.random_normal([batch_size,50,1],0.5,1/50))
    b1 = tf.Variable(tf.constant(0.5,shape = [50]))
    b2 = tf.Variable(tf.constant(0.3,shape = [1]))
    first_out = tf.nn.tanh(tf.add(tf.matmul(combined_BiRnn_output,Wx1),b1))
    e = tf.nn.relu(tf.add(tf.matmul(first_out,Wx2),b2))
        
    return e

In [11]:
def softmax(e_values):
    alphas = tf.exp(e_values)
    totals = tf.reduce_sum(tf.exp(e_values),1)
    alphas = tf.unstack(alphas,axis = 0)
    totals = tf.unstack(totals, axis = 0)
    for i in range(batch_size):
        alphas[i] = alphas[i]/totals[i]
    alphas = tf.stack(alphas,axis = 0)
    return alphas

In [12]:
e_vals = neural_network(combined_BiRNN_outputs)
alpha_values = softmax(e_vals)

In [13]:
init = tf.global_variables_initializer()

** To test whether alpha values are coming or not **

In [14]:
with tf.Session() as sess:
    sess.run(init)
    batch_x, v = next_batch(articles,headings,batch_size)
    outs = sess.run(outputs,feed_dict = {X: batch_x})
    
    alpha = sess.run(alpha_values,feed_dict = {combined_BiRNN_outputs:outs})

** Weighted Average **

In [4]:
weighted_average = outputs*alpha_values

NameError: name 'outputs' is not defined